In [18]:
import pandas as pd
import numpy as np
import pickle
import time 
import os
import re
from datetime import datetime


In [19]:
# load
sdgd_df = pd.read_csv('cleaned.CSV', encoding="gbk",index_col = 0)



In [1]:
# NLP算法
# 对每个数据，生成[代码，第几投资者，投资者名字，所属单位名字]
# 除了所属公司，其他数据都是excel提供的，算法主要用于提取所属单位名字
# 流程 
# 1 取纯英文 统一格式后保存为所属公司
# 2 取短的名字（三字以下）， 直接保存为所属单位
# 3 通过split，一组以下的，直接保存为所属单位，一组以上的，通过语料库查找，match到关键词的，通过关键词生成所属单位
# 4 match不到关键词的，第一个组成部为保存为所属单位
# 5 中文语料库是通过一天的数据生成的，覆盖率较高，但是也会有一些遗漏，后续维护需要根据往日数据，或者看结果感觉不对的，添加语料库


# 统一一下格式：wind数据不规范
sdgd_df.replace("—","-",inplace = True)
sdgd_df.replace("·","·",inplace = True)
    
nmlst = ['n'+str(i) for i in range(1,11)]
    
name_pool = []
for code in sdgd_df.columns:
    for nm in nmlst:
        if sdgd_df[code][nm] != '0' and sdgd_df[code][nm] != 0 and sdgd_df[code][nm] != 0.0 :
            name_pool.append(sdgd_df[code][nm])
# 所有名字
name_unique = list(set(name_pool))   


NameError: name 'sdgd_df' is not defined

In [33]:
# 下面的是block代码，一个个run就能用
name_pool = []
for code in sdgd_df.columns:
    for nm in nmlst:
        if sdgd_df[code][nm] != '0' and sdgd_df[code][nm] != 0 and sdgd_df[code][nm] != 0.0 :
            name_pool.append(sdgd_df[code][nm])

name_unique = list(set(name_pool))   

# DGD = dict()
# for code in sdgd_df.columns:
#     DGD[code] = dict()
# for code in 

In [34]:
# find all English
cn = []
eg = []
def con_cn(strs):
    for _char in strs:
        if '\u4e00' <= _char <= '\u9fa5':
            return True
    return False


def con_eg(st):
    return  bool(re.search('[a-zA-z]', st))

eg = []
for nm in name_unique:
    if not con_cn(nm):
        eg.append(nm)
        print(nm)
    
    else:
        cn.append(nm)
print(len(eg))
print(len(cn))

NEW FORTUNE INTERNATIONAL GROUP LTD.
UBS   AG
JPMORGAN CHASE BANK,NATIONAL ASSOCIATION
MORGAN STAN LEY & CO. I NTERNATIONA L PLC.
ALPHA HOLDING VENTURES  LIMITED
YU JI QUN
BNP Paribas Lease Group
LU XIANPING
UBSAG
MORGANSTANLEY&CO.INTERNATIONALPLC
UBS AG
MORGAN STANLEY & CO. INTERNATIONAL PLC.
JPMORGAN CHASE BANK, NATIONAL ASSOCIATION
UBS  AG
Hong Leong Bank Berhad
JPMORGANCHASEBANK,NATIONALASSOCIATION
Universal Dairy
MERRILL LYNCH INTERNATIONAL
LUO DONGPING
19
1785


In [23]:

eg_ns = []
for one in eg:
    new_one = [s for s in one if s.isalpha()]
    new_str = ''
    for s in new_one:
        new_str += s
        
    eg_ns.append(new_str)
    
eg_ns = list(set(eg_ns))
print(len(eg_ns))
print(eg_ns)
print(eg)
        
    
clean = []
for one in eg_ns:
    candi = ''
    for name in eg:

        new_one = [s for s in name if s.isalpha()]
        new_str = ''
        for s in new_one:
            new_str += s
        
        if one == new_str:
#             print(one,new_str,name)
            if candi == '':
                candi = name
            else:
                if len(name) < len(candi):
                    candi = name
    
    
    clean.append(candi)

#最终结果
clean

12
['JPMORGANCHASEBANKNATIONALASSOCIATION', 'MERRILLLYNCHINTERNATIONAL', 'LUODONGPING', 'MORGANSTANLEYCOINTERNATIONALPLC', 'LUXIANPING', 'NEWFORTUNEINTERNATIONALGROUPLTD', 'ALPHAHOLDINGVENTURESLIMITED', 'UniversalDairy', 'UBSAG', 'HongLeongBankBerhad', 'BNPParibasLeaseGroup', 'YUJIQUN']
['NEW FORTUNE INTERNATIONAL GROUP LTD.', 'UBS   AG', 'JPMORGAN CHASE BANK,NATIONAL ASSOCIATION', 'MORGAN STAN LEY & CO. I NTERNATIONA L PLC.', 'ALPHA HOLDING VENTURES  LIMITED', 'YU JI QUN', 'BNP Paribas Lease Group', 'LU XIANPING', 'UBSAG', 'MORGANSTANLEY&CO.INTERNATIONALPLC', 'UBS AG', 'MORGAN STANLEY & CO. INTERNATIONAL PLC.', 'JPMORGAN CHASE BANK, NATIONAL ASSOCIATION', 'UBS  AG', 'Hong Leong Bank Berhad', 'JPMORGANCHASEBANK,NATIONALASSOCIATION', 'Universal Dairy', 'MERRILL LYNCH INTERNATIONAL', 'LUO DONGPING']


['JPMORGANCHASEBANK,NATIONALASSOCIATION',
 'MERRILL LYNCH INTERNATIONAL',
 'LUO DONGPING',
 'MORGANSTANLEY&CO.INTERNATIONALPLC',
 'LU XIANPING',
 'NEW FORTUNE INTERNATIONAL GROUP LTD.',
 'ALPHA HOLDING VENTURES  LIMITED',
 'Universal Dairy',
 'UBSAG',
 'Hong Leong Bank Berhad',
 'BNP Paribas Lease Group',
 'YU JI QUN']

In [24]:
short = []
long = []
for one in cn:
    if len(one) <=3:
        short.append(one)
    else:    
        long.append(one)
        
print(len(long))

1061


In [25]:
long_more = []
one_grp = []
for nm in long:
    nms = re.split('-|·|·|—',nm)
    if len(nms) == 1:
        one_grp.append(nm)
    else:
        long_more.append(nm)

In [27]:
print(len(one_grp))
print(len(long_more))
# 一组数据最终结果
one_grp


401
660


['常熟市发展投资有限公司',
 '全国社保基金一一四组合',
 '华友控股集团有限公司',
 '基本养老保险基金一零七组合',
 '北京山通投资咨询有限责任公司',
 '登记结算系统债券回购质押专用账户(北京银行股份有限公司)',
 '江苏飞翔化工股份有限公司',
 '云南大业盛德企业管理有限公司',
 '江西水泥有限责任公司',
 '济南漱玉锦云投资合伙企业(有限合伙)',
 '国海证券股份有限公司',
 '登记结算系统债券回购质押专用账户(中国工商银行)',
 '广州萝岗区汇资投资管理合伙企业(有限合伙)',
 '全国社保基金二零六组合',
 '众兴集团有限公司',
 '中信建投证券股份有限公司客户信用交易担保证券账户',
 '济南西城置业有限公司',
 '芜湖富春创业投资合伙企业(有限合伙)',
 '洋浦天原投资有限公司',
 '宁波盛泰纺织有限公司',
 '北京安东企业管理合伙企业(有限合伙)',
 '全国社保基金一零一组合',
 '深圳市信庭至美半导体企业(有限合伙)',
 '杭州市财开投资集团有限公司',
 '御投投资',
 '青岛煜明投资中心(有限合伙)',
 '中银国际证券股份有限公司',
 '基本养老保险基金一零二组合',
 '南方希望实业有限公司',
 '御家投资',
 '基本养老保险基金一五零三二组合',
 '上海中件管理咨询有限公司',
 '江苏清海投资有限公司',
 '三峡资本控股有限责任公司',
 '苏州熔拓创新创业投资企业(有限合伙)',
 '登记结算系统债券回购质押专用账户(交通银行)',
 '烟台南晟投资合伙企业(有限合伙)',
 '浙江恒逸集团有限公司',
 '太平洋证券股份有限公司',
 '济南市国有资产运营有限公司',
 '广州立创企业管理咨询合伙企业(有限合伙)',
 '江苏广靖锡澄高速公路有限责任公司',
 '仙游泓祥企业管理合伙企业(有限合伙)',
 '高金富恒集团有限公司',
 '基本养老保险基金一零五组合',
 '御投贰号',
 '浙江金帆达生化股份有限公司',
 '宁波利时信息科技有限公司',
 '湖南艾华控股有限公司',
 '广西柳工集团有限公司',
 '全国社保基金二一三组合',
 '福建省财政厅',
 '北京融拓智慧农业投资合伙企业(有限合伙)',
 '天合星元投资发展有限公司',
 '深圳泰巨科技投资管理

In [28]:
# 名字存到这里 这个很重要！别删掉！是我手动加的
jg = ['全国社保', '基本养老保险','博时','前海','银河','信诚','朱雀','睿远','中信建投','创金合信','华宝','宝康','海富通','泰康','东方','建信','浙商','中庚','长信','汇安','光大保德信','光大','宝盈','鹏华','久赢','兴全','华富','融通','易方达','中海','财通','金信', '中欧','御投','国寿','兴业','信澳','国投瑞银','工银瑞信','博道','天弘','诺安','国泰','广发','鹏华','华夏','景顺长城','汇添富','大成','泓德','中银','富国','中金','金鹰','招商','南方','格林','华安','华泰','西部','安信','交银施罗德','信澳','嘉实','浦银安盛','太平','中邮','银华','中信','平安','华商','华富'] 

long_extract = []
long_ne = []
for nm in long_more:
    flag = 0
    for j in jg:
        if j in nm:
            if j+'银行' not in nm:
                if j+'期货' not in nm:
                    if j+'证券' not in nm:
                        if j+'保险' not in nm:
                            if j+'股份' not in nm:
                                if j+'人寿' not in nm:
                                    flag = 1
                                    if j == '中信' and '中信建投' in nm:
                                        flag = 0
                                        
                                    if '中信期货' in nm:
                                        flag = 0
                                    if j == '金信' and '资金信' in nm:
                                        flag = 0
                                    
                                    if flag == 1:
                                        print(j+'基金', nm) 
                                        long_extract.append((j+'基金',nm))
                                        
                                        
                                        
    if flag == 0:
        long_ne.append(nm)
#最终对应结果
print(len(long_ne))
print(len(long_extract))
        
# 中信和中信建投        
#     for j in jg:
#         print(j,nm)
#         if j in nm:
#             if j+'银行' not in nm:


富国基金 中国工商银行股份有限公司-富国收益增强债券型证券投资基金
国泰基金 中国建设银行股份有限公司-国泰可转债债券型证券投资基金
太平基金 太平养老金溢丰固定收益型养老金产品-中国工商银行股份有限公司
前海基金 深圳前海卓亿资产管理有限公司-卓亿稳鑫多策略3号私募证券投资基金
易方达基金 招商银行股份有限公司-易方达裕鑫债券型证券投资基金
光大保德信基金 中国建设银行股份有限公司-光大保德信增利收益债券型证券投资基金
光大基金 中国建设银行股份有限公司-光大保德信增利收益债券型证券投资基金
大成基金 大成基金管理有限公司-社保基金17011组合
交银施罗德基金 中国建设银行股份有限公司-交银施罗德增利增强债券型证券投资基金
国寿基金 国寿养老稳健5号固定收益型养老金产品-中国工商银行股份有限公司
中金基金 中金安胜固定收益型养老金产品-中国建设银行股份有限公司
工银瑞信基金 中国建设银行-工银瑞信信用添利债券型证券投资基金
博时基金 博时价值回报固定收益型养老金产品-中国银行股份有限公司
富国基金 富国富益进取固定收益型养老金产品-中国工商银行股份有限公司
泰康基金 泰康资产鑫全·多策略2号固定收益型养老金产品-中国工商银行股份有限公司
鹏华基金 北京银行股份有限公司-鹏华双债加利债券型证券投资基金
鹏华基金 北京银行股份有限公司-鹏华双债加利债券型证券投资基金
金鹰基金 中国工商银行股份有限公司-金鹰元丰债券型证券投资基金
易方达基金 易方达稳健配置混合型养老金产品-中国工商银行股份有限公司
华夏基金 华夏基金延年益寿4号固定收益型养老金产品-中国农业银行股份有限公司
工银瑞信基金 中国农业银行股份有限公司-工银瑞信战略转型主题股票型证券投资基金
兴全基金 兴全基金-兴业银行-兴业证券股份有限公司
银华基金 银华坤利3号固定收益型养老金产品-招商银行股份有限公司
信澳基金 中国工商银行股份有限公司-信澳智远三年持有期混合型证券投资基金
信澳基金 中国工商银行股份有限公司-信澳智远三年持有期混合型证券投资基金
易方达基金 中国银行-易方达平稳增长证券投资基金
富国基金 中国建设银行股份有限公司-富国天丰强化收益债券型证券投资基金
中邮基金 中邮理财有限责任公司-邮银财富·鸿锦周期270天型1号
嘉实基金 嘉实元麒混合型养老金-中国建设银行股份有限公司
博时基金 

In [31]:
long_ne
for nm in long_ne:
    nms = re.split('-|·|·|—',nm)
    print((nms[0], nm))

('百年保险资管', '百年保险资管-宁波银行-百年资管弘远22号资产管理产品')
('中意资管', '中意资管-工商银行-中意资产-债券优选资产管理产品')
('贝莱德(新加坡)有限公司', '贝莱德(新加坡)有限公司-贝莱德全球基金-中国债券基金(交易所)')
('广州守正用奇私募基金管理有限公司', '广州守正用奇私募基金管理有限公司-中昕稳健1号私募证券投资基金')
('中信期货', '中信期货-兴业银行-中信期货睿丰1号集合资产管理计划')
('招商银行股份有限公司', '招商银行股份有限公司-富兰克林国海恒瑞债券型证券投资基金')
('华能贵诚信托有限公司', '华能贵诚信托有限公司-华能信托·元诚1号集合资金信托计划')
('广东一叶私募基金管理合伙企业(有限合伙)', '广东一叶私募基金管理合伙企业(有限合伙)-广东一叶价值成长私募证券投资基金')
('西北投资管理(香港)有限公司', '西北投资管理(香港)有限公司-西北飞龙基金有限公司')
('华泰证券资管', '华泰证券资管-华夏人寿保险股份有限公司-华泰华夏人寿3号单一资产管理计划')
('九坤投资(北京)有限公司', '九坤投资(北京)有限公司-九坤策略智选30号私募证券投资基金')
('青岛上风私募基金管理有限公司', '青岛上风私募基金管理有限公司-上风强债1号私募证券投资基金')
('北京乐雪私募基金管理合伙企业(有限合伙)', '北京乐雪私募基金管理合伙企业(有限合伙)-乐雪私募证券投资基金')
('上海千宜投资管理中心(有限合伙)', '上海千宜投资管理中心(有限合伙)-千宜定制22号私募证券投资基金')
('福州开发区谱成资产管理有限公司', '福州开发区谱成资产管理有限公司-谱成本固1号私募证券投资基金')
('中泰证券资管', '中泰证券资管-招商银行-中泰星河12号集合资产管理计划')
('中信证券信福晚年固定收益型养老金产品', '中信证券信福晚年固定收益型养老金产品-中国建设银行股份有限公司')
('上海宁泉资产管理有限公司', '上海宁泉资产管理有限公司-宁泉致远18号私募证券投资基金')
('上海喜世润投资管理有限公司', '上海喜世润投资管理有限公司-喜世润飞岳3号私募证券投资基金')
('大家资产', '大家资产-民生银行-大家资产-稳健精选3号(第五期)集合资

In [ ]:
这里就结束了, 一共分了五类,纯英文类,三字及以下类,split一组类,split多组match关键字类,split多组无match类,每一类的所属单位都已经提取.


In [32]:
# for nm in long:
#     nms = re.split('-|·|·|—',nm)
#     if len(nms) == 1:
#         print(nms)
#     count = 0
#     for subnm in nms:
#         if '公司' in subnm or '企业' in subnm:
#             count += 1 
    
#     if count != 1:
#         print(count, nms)

#平安，中信。兴业是个问题，又是银行又是基金，可以考虑抓两个关键字或者单纯按顺序来
#抓平安银行，中信银行，兴业银行,招商银行
#上证红利交易型开放式指数证券投资基金
#中证500交易型开放式指数证券投资基金
# 中信建投 和中信
# jg = ['全国社保', '基本养老保险','博时','前海','银河','信诚','朱雀','睿远','中信建投','创金合信','华宝','宝康','海富通','泰康','东方','建信','浙商','中庚','长信','汇安','光大保德信','光大','宝盈','鹏华','久赢','兴全','华富','融通','易方达','中海','财通','金信', '中欧','御投','国寿','兴业','信澳','国投瑞银','工银瑞信','博道','天弘','诺安','国泰','广发','鹏华','华夏','景顺长城','汇添富','大成','泓德','中银','富国','中金','金鹰','招商','南方','格林','华安','华泰','西部','安信','交银施罗德','信澳','嘉实','浦银安盛','太平','中邮','银华','中信','平安','华商','华富'] 

# count = 0
# for nm in long:

#     nms = re.split('-|·|·|—',nm)
    

    
#     if count > 750 and len(nms) != 1:
#         print(nms)
#     count += 1

In [68]:
print(len(long))

1061
